<div>
<img src="../image/images.png" width="100%"/>
</div>

<h1>
    <center>
        H64 Data Product
        <br>
        <br>
        Analysis
    </center>
</h1>

## Introduction 
The H64 product provides high-resolution satellite-derived rainfall estimates that are essential for understanding precipitation patterns and their impacts. This analysis focuses on leveraging the H64 dataset to extract meaningful insights into rainfall distribution, frequency, and temporal trends over a defined area of interest (AOI) and period.

The objectives of this analysis include:  
- Visualizing spatial and temporal rainfall variability.  
- Performing statistical evaluations to characterize the rainfall dataset.  
- Highlighting patterns and anomalies that inform hydrological and climatological studies.  

By combining advanced data visualization techniques with statistical tools, this analysis aims to provide actionable insights into the behavior of precipitation across the study area. The methodologies and results presented here can support decision-making processes in various fields, including water resource management, agriculture, and disaster risk assessment.  

In the following sections, you will find detailed explorations of the H64 rainfall data, beginning with its visualization and progressing through statistical and frequency analyses.

### Libraries import
This code cell imports the necessary libraries and modules required for the analysis of the H64 Accumulated Precipitation data product. These libraries enable data visualization and statistical evaluation, ensuring a seamless workflow for geospatial and temporal analysis.

In [ ]:
import os
from datetime import datetime, date

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd
from IPython.display import display, IFrame
from shapely.geometry import Polygon
from scipy.stats import skew
import matplotlib.dates as mdates

from hsaf_data_access import HSAFDataAccess as data_access

### Data Loading  

This cell identifies and loads the H64 Accumulated Precipitation data files from the `data` directory. It checks for available NetCDF files, sorts them for proper sequencing, and combines them into a single dataset using `xarray`. This ensures the data is ready for analysis across multiple files.

In [ ]:
# list the files in the data path and sort them
filelist = [f for f in os.listdir('data') if f.endswith('.nc')]
# Check if the filelist is empty or None
if not filelist:
    raise ValueError("No files found in the directory: {}".format(data_path))
else:
    filelist.sort()
    
# Open the files for analysis
ds = xr.open_mfdataset([os.path.join('data', f) for f in filelist], combine='by_coords', engine='netcdf4')

### Total Precipitation Analysis  

This section calculates the total accumulated rainfall (`acc_rr`) over the entire study period by summing data across the time dimension. The results are visualized as a spatial map, highlighting the distribution of total rainfall across the region. The map uses a color scale to represent precipitation intensity, with values labeled in millimeters (mm). The figure is saved as an image for reference.

In [ ]:
total_rr = ds['acc_rr'].sum(dim='time', skipna=True)
    
tt = total_rr.plot(
        x='longitude',
        y='latitude',
        cmap='rainbow',
        cbar_kwargs={"label": "Total Rain rates [mm]", "pad": 0.05},
        vmin=total_rr.min().compute(),
        vmax=total_rr.max().compute()
    )

plt.title('Total Precipitation Estimates', fontsize=12.5)

# Save the figure
plt.savefig('output/H64_Total_Rainfall.png', dpi=300)

Assuming the AOI is provided as a shapefile or a shapefile is available, this cell focuses on overlaying it on the plot.

1. **Generate AOI Selection Widget**:  
   - The `create_shp_widgets` method from `data_access` generates an interactive widget, allowing users to select a shapefile representing the AOI.

2. **Read AOI Shapefile**:  
   - Reads the selected shapefile into a GeoDataFrame (`aoi_gdf`) using `geopandas`. This GeoDataFrame represents the boundaries or geometry of the AOI.

3. **Overlay AOI on Precipitation Plots**:  
   - Iterates over each subplot (`ax`) in the precipitation plots and overlays the AOI boundaries (`aoi_gdf.boundary`) using black lines for clear visual delineation of the region.

4. **Save the Updated Visualization**:  
   - Exports the updated precipitation plot with the AOI overlay as a high-resolution image file for documentation or further analysis.

In [ ]:
aoi_shp = data_access.create_shp_widgets()
aoi_shp

In [ ]:
aoi_gdf = gpd.read_file(aoi_shp.children[0].value)
# Plot the AOI boundary on top of the plot
aoi_gdf.boundary.plot(ax=tt.axes, color='black', linewidth=0.1)

tt.figure.savefig('output/H64_Total_Rainfall.png', dpi=300)

### Mean Precipitation Analysis  

This section computes the mean rainfall (`acc_rr`) over the study period by averaging the data along the time dimension. The results are displayed as a spatial map, showing the average precipitation intensity in millimeters (mm) across the region. This visualization highlights areas with consistent rainfall patterns. The figure is saved for documentation purposes.

In [ ]:
mean_rr = ds['acc_rr'].mean(dim='time', skipna=True)

tt_mean = mean_rr.plot(
            x='longitude',
            y='latitude',
            cmap='rainbow',
            cbar_kwargs={"label": "Mean Rain rates [mm]", "pad": 0.05},
            vmin=mean_rr.min().compute(),
            vmax=mean_rr.max().compute()
        )
    
plt.title('Mean Precipitation Estimates', fontsize=12.5)

# Save the figure
plt.savefig('output/H64_Mean_Rainfall.png', dpi=300)



In [ ]:
aoi_shp = data_access.create_shp_widgets()
aoi_shp

In [ ]:
aoi_gdf = gpd.read_file(aoi_shp.children[0].value)
# Plot the AOI boundary on top of the plot
aoi_gdf.boundary.plot(ax=tt_mean.axes, color='black', linewidth=0.1)

tt_mean.figure.savefig('output/H64_Mean_Rainfall.png', dpi=300)

### Daily Mean Rainfall Time Series  

This section evaluates the temporal variation of rainfall over the study period by calculating the daily mean precipitation (`acc_rr`) across all spatial dimensions. The resulting time series plot provides insights into rainfall trends, seasonal patterns, and any significant fluctuations. If only one file is available, a time series plot cannot be generated. The figure is saved for further reference.

In [ ]:
if len(filelist) == 1:
    print(f'Cannot produce time plot for a day\'s file')
else:
    # Calculate daily mean rainfall over the spatial dimensions
    mean_per_day = ds['acc_rr'].mean(dim=['longitude', 'latitude'], skipna=True)
    mean_per_day['time'].dt
    mean_per_day.plot(marker='o', linestyle='-', color='blue', label='Mean Rainfall')

    # # Add labels and title
    plt.title('Daily Mean Rainfall Time Series', fontsize=12.5)
    plt.xlabel('Day', fontsize=12)
    plt.ylabel('Mean Rain rates [mm]', fontsize=12)
    plt.savefig('output/H64_Time_Series.png', dpi=300)

### Rainfall Frequency Distribution  

This code analyzes the frequency distribution of accumulated rainfall data by dividing it into 10 bins based on its range. The histogram visualizes the distribution on a logarithmic scale, highlighting patterns across different rainfall intensities. This analysis helps to understand the occurrence of various rainfall rates over the study period. The resulting figure is saved for future reference.

In [ ]:
# Flatten the accumulated rainfall data
rainfall_data_flat = ds['acc_rr'].values.flatten()

rainfall_data_flat = rainfall_data_flat[~np.isnan(rainfall_data_flat)]

# Find the minimum and maximum values for binning
rainfall_min = np.min(rainfall_data_flat)
rainfall_max = np.max(rainfall_data_flat)

# Number of bins for the histogram
num_bins = 10
bins = np.linspace(rainfall_min, rainfall_max, num_bins)

# Calculate the frequency distribution
rainfall_freq, bin_edges = np.histogram(rainfall_data_flat, bins=bins)

# Plot with Logarithmic Y-axis
plt.figure(figsize=(10, 6))
plt.bar(bin_edges[:-1], rainfall_freq, width=np.diff(bin_edges), edgecolor="black", align="edge", color='skyblue')
plt.yscale('log')

plt.title('Rainfall Frequency Distribution (Log Scale)', fontsize=12.5)
plt.xlabel('Rainfall (mm)', fontsize=12)
plt.ylabel('Frequency (Log Scale)', fontsize=12)
plt.xticks(bin_edges,)

plt.savefig('output/H64_log_freq.png', dpi=300)

### Statistical Analysis of Rainfall

This code computes key statistical metrics for the accumulated rainfall data, including the mean, median, standard deviation, and skewness. These statistics provide a summary of the rainfall distribution for the defined period and Area of Interest (AOI). The results help assess the central tendency, variability, and asymmetry of the rainfall data. The calculated statistics are printed to the console and saved in a text file for further analysis.

In [ ]:
mean_rainfall = np.mean(rainfall_data_flat)
median_rainfall = np.median(rainfall_data_flat)
std_dev_rainfall = np.std(rainfall_data_flat, ddof=1)  # Sample standard deviation
skewness_rainfall = skew(rainfall_data_flat)

statistics = f"""Rainfall Statistics for the defined period and AOI:
----------------------
Overall Mean Rainfall:          {mean_rainfall:.2f} mm
Median Rainfall:                {median_rainfall:.2f} mm
Standard Deviation of Rainfall: {std_dev_rainfall:.2f} mm
Skewness of Rainfall:           {skewness_rainfall:.2f}
"""

print(statistics)

with open('output/rainfall_statistics.txt', 'w') as f:
    f.write(statistics)

<table style="width: 100%; border-collapse: collapse;">
    <tr>
        <td style="text-align: left; font-size: 15px;">
            <a href="data_visualisation.ipynb" style="text-decoration:none;">&larr; Data Visualisation </a>
        </td>
        <td style="text-align: right; font-size: 15px;">
            <a href="case_study.ipynb" style="text-decoration:none;"> Sample case study &rarr;</a>
        </td>
    </tr>
</table>